In [10]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
sv1849 = [283, 157, 192, 249, 259, 226, 352, 97, 111, 8, 235, 92]
lsv1849 = [256, 267, 312, 257, 318, 446, 143, 193, 243, 215, 544, 187, 153, 81, 113, 176]
sv1854 = [371, 161, 148, 362, 244, 237, 282, 59, 171, 9, 240, 174]
lsv1854 = [113, 174, 270, 93, 210, 388, 92, 58, 117, 49, 193, 303, 142, 48, 165, 132]

### 前後比較結果を比較する

In [36]:
a = sum(sv1854) - sum(sv1849) # sv対象地域の固有の効果を消す
b = sum(lsv1854) - sum(lsv1849) # lsv対象地域の固有の効果を消す
print(b - a) # 平行トレンド過程でどちらの地域も時間による固有の効果は同じと仮定し、その効果を消す

-1554

### 回帰分析で効果検証

In [38]:
JS_sum = pd.DataFrame({
    "y" : [sum(sv1849), sum(lsv1849), sum(sv1854), sum(lsv1854)],
    "lsv" : [0,1,0,1], # 地域の固有の効果を表している
    "d54" : [0,0,1,1], # 時間による固有の効果を表している
    "lsv*d54":[0,0,0,1] # 1854年に介入があった地域のデータであることを示す(知りたい効果量)
})
"""
sum(lsv1854) - sum(lsv1849) = (1*lsv + 1*d54 + 1*lsv*d54) - (1*lsv + 0*d54 + 0*lsv*d54)
                            = (1*d54 + 1*lsv*d54) # 地域固有の効果が消えた
sum(sv1854) - sum(sv1849) = (0*lsv + 1*d54 + 0*lsv*d54) - (0*lsv + 0*d54 + 0*lsv*d54)
                            = (1*d54) # 地域固有の効果が消えた
(1*d54 + 1*lsv*d54) - (1*d54) = 1*lsv*d54 # なのでlsv*d54の係数が効果量を表している
"""

JS_sum.head()

,y,lsv,d54,lsv*d54
0,2261,0,0,0
1,3904,1,0,0
2,2458,0,1,0
3,2547,1,1,1


In [39]:
y = JS_sum[["y"]]
X = JS_sum[["lsv", "d54", "lsv*d54"]]
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                     0.000
Date:                Tue, 16 Aug 2022   Prob (F-statistic):                nan
Time:                        21:58:44   Log-Likelihood:                 104.68
No. Observations:                   4   AIC:                            -201.4
Df Residuals:                       0   BIC:                            -203.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2261.0000        inf          0        nan         nan         nan
lsv         1643.0000        inf          0        nan         nan         nan
d54          197.0000        inf          0        nan         nan         nan
lsv*d54    -1554.0000        inf         -0        nan         nan         nan
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   0.857
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.348
Skew:                          -0.435   Prob(JB):                        0.840
Kurtosis:                       1.846   Cond. No.                         6.85
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
JS_df = pd.DataFrame({
    "y" : sv1849+sv1854+lsv1849+lsv1854,
    "area" : [f"Area{i+1}" for i in range(len(sv1849))]*2 + [f"Area{len(sv1849)+i+1}" for i in range(len(lsv1849))]*2,
    "lsv" : [0]*len(sv1849)*2 + [1]*len(lsv1849)*2,
    "d54" : [0]*len(sv1849) + [1]*len(sv1854) + [0]*len(lsv1849) + [1]*len(lsv1854),
    "lsv*d54":[0]*len(sv1849+sv1854+lsv1849) + [1]*len(lsv1854)
})
JS_df = pd.concat([JS_df, pd.get_dummies(JS_df["area"], drop_first=True)], axis=1)
JS_df.head()

,y,area,lsv,d54,lsv*d54,Area10,Area11,Area12,Area13,Area14,...,Area26,Area27,Area28,Area3,Area4,Area5,Area6,Area7,Area8,Area9
0,283,Area1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,157,Area2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,192,Area3,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,249,Area4,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,259,Area5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [34]:
y = JS_df[["y"]]
X = JS_df.drop(["y", "area"], axis=1)
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.845
Model:                            OLS   Adj. R-squared:                  0.672
Method:                 Least Squares   F-statistic:                     4.881
Date:                Mon, 15 Aug 2022   Prob (F-statistic):           5.45e-05
Time:                        19:02:36   Log-Likelihood:                -289.21
No. Observations:                  56   AIC:                             638.4
Df Residuals:                      26   BIC:                             699.2
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        318.7917     45.727      6.972      0.000     224.799     412.785
lsv          -70.3922     45.452     -1.549      0.134    -163.821      23.036
d54           16.4167     25.365      0.647      0.523     -35.721      68.555
lsv*d54     -101.2292     33.554     -3.017      0.006    -170.201     -32.257
Area10      -318.5000     62.131     -5.126      0.000    -446.211    -190.789
Area11       -89.5000     62.131     -1.441      0.162    -217.211      38.211
Area12      -194.0000     62.131     -3.122      0.004    -321.711     -66.289
Area13       -21.4933     42.633     -0.504      0.618    -109.126      66.140
Area14        14.5067     42.633      0.340      0.736     -73.126     102.140
Area15        85.0067     42.633      1.994      0.057      -2.626     172.640
Area16       -30.9933     42.633     -0.727      0.474    -118.626      56.640
Area17        58.0067     42.633      1.361      0.185     -29.626     145.640
Area18       211.0067     42.633      4.949      0.000     123.374     298.640
Area19       -88.4933     42.633     -2.076      0.048    -176.126      -0.860
Area2       -168.0000     62.131     -2.704      0.012    -295.711     -40.289
Area20       -80.4933     42.633     -1.888      0.070    -168.126       7.140
Area21       -25.9933     42.633     -0.610      0.547    -113.626      61.640
Area22       -73.9933     42.633     -1.736      0.094    -161.626      13.640
Area23       162.5067     42.633      3.812      0.001      74.874     250.140
Area24        39.0067     42.633      0.915      0.369     -48.626     126.640
Area25       -58.4933     42.633     -1.372      0.182    -146.126      29.140
Area26      -141.4933     42.633     -3.319      0.003    -229.126     -53.860
Area27       -66.9933     42.633     -1.571      0.128    -154.626      20.640
Area28       -51.9933     42.633     -1.220      0.234    -139.626      35.640
Area3       -157.0000     62.131     -2.527      0.018    -284.711     -29.289
Area4        -21.5000     62.131     -0.346      0.732    -149.211     106.211
Area5        -75.5000     62.131     -1.215      0.235    -203.211      52.211
Area6        -95.5000     62.131     -1.537      0.136    -223.211      32.211
Area7        -10.0000     62.131     -0.161      0.873    -137.711     117.711
Area8       -249.0000     62.131     -4.008      0.000    -376.711    -121.289
Area9       -186.0000     62.131     -2.994      0.006    -313.711     -58.289
==============================================================================
Omnibus:                        6.390   Durbin-Watson:                   2.028
Prob(Omnibus):                  0.041   Jarque-Bera (JB):               10.433
Skew:                          -0.000   Prob(JB):                      0.00543
Kurtosis:                       5.115   Cond. No.                     3.46e+16
==

In [40]:
# 目的変数の対数を取り、係数に何%減少したかの解釈を与える
y = JS_df[["y"]]
y = np.log(y)
X = JS_df.drop(["y", "area"], axis=1)
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.919
Model:                            OLS   Adj. R-squared:                  0.830
Method:                 Least Squares   F-statistic:                     10.23
Date:                Tue, 16 Aug 2022   Prob (F-statistic):           3.07e-08
Time:                        22:01:46   Log-Likelihood:                 4.8197
No. Observations:                  56   AIC:                             50.36
Df Residuals:                      26   BIC:                             111.1
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.7439      0.240     23.952      0.000       5.251       6.237
lsv           -0.3337      0.238     -1.400      0.173      -0.824       0.156
d54            0.0739      0.133      0.556      0.583      -0.200       0.347
lsv*d54       -0.5661      0.176     -3.217      0.003      -0.928      -0.204
Area10        -3.6425      0.326    -11.179      0.000      -4.312      -2.973
Area11        -0.3107      0.326     -0.954      0.349      -0.980       0.359
Area12        -0.9404      0.326     -2.886      0.008      -1.610      -0.271
Area13        -0.0278      0.224     -0.124      0.902      -0.487       0.432
Area14         0.2091      0.224      0.935      0.358      -0.250       0.669
Area15         0.5066      0.224      2.266      0.032       0.047       0.966
Area16        -0.1232      0.224     -0.551      0.586      -0.583       0.336
Area17         0.3905      0.224      1.747      0.093      -0.069       0.850
Area18         0.8666      0.224      3.876      0.001       0.407       1.326
Area19        -0.4218      0.224     -1.886      0.070      -0.881       0.038
Area2         -0.7120      0.326     -2.185      0.038      -1.382      -0.042
Area20        -0.5025      0.224     -2.248      0.033      -0.962      -0.043
Area21        -0.0365      0.224     -0.163      0.872      -0.496       0.423
Area22        -0.5328      0.224     -2.383      0.025      -0.992      -0.073
Area23         0.6167      0.224      2.759      0.010       0.157       1.076
Area24         0.3083      0.224      1.379      0.180      -0.151       0.768
Area25        -0.1709      0.224     -0.765      0.451      -0.631       0.289
Area26        -1.0312      0.224     -4.612      0.000      -1.491      -0.572
Area27        -0.2474      0.224     -1.107      0.279      -0.707       0.212
Area28        -0.1374      0.224     -0.615      0.544      -0.597       0.322
Area3         -0.6535      0.326     -2.006      0.055      -1.323       0.016
Area4         -0.0763      0.326     -0.234      0.817      -0.746       0.593
Area5         -0.2538      0.326     -0.779      0.443      -0.924       0.416
Area6         -0.3365      0.326     -1.033      0.311      -1.006       0.333
Area7         -0.0281      0.326     -0.086      0.932      -0.698       0.642
Area8         -1.4547      0.326     -4.465      0.000      -2.124      -0.785
Area9         -0.8552      0.326     -2.625      0.014      -1.525      -0.185
==============================================================================
Omnibus:                        0.064   Durbin-Watson:                   1.761
Prob(Omnibus):                  0.969   Jarque-Bera (JB):                0.017
Skew:                           0.000   Prob(JB):                        0.992
Kurtosis:                       2.915   Cond. No.                     3.46e+16
==